In [ ]:
#导入模块
import  numpy as np # numpy 库用于矩阵运算
import pandas as pd # pandas 库用于读取数据
from   pandas import DataFrame
import plotly # plotly 库用于绘制动态图形
import plotly.express as px # plotly.express 库用于绘制动态图形
import plotly.graph_objects as go # plotly.graph_objects 库用于绘制动态图形
import baostock as bs # 用于获取股票数据
import ydata_profiling # 用于数据探索
from ydata_profiling import * # 用于数据探索
import datetime # 用于时间处理
import time # 用于时间处理


In [ ]:
#### 获取沪深A股历史K线数据 ####
# 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
# 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
# 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg
# 以下函数默认获取上证综合指数从2023-01-01开始，到2023-01-31结束的日K线数据，仅用作测试接口是否正常。正式使用时，可以调整get_stock_data函数的参数。
# 其中code为股票代码，默认为sh.000001，即上证综合指数。start_date为开始日期，默认为2023-01-01。end_date为结束日期，默认为2023-01-31。

def get_stock_data(code='sz.000002', index:str = "" , start_date = '2023-01-01', end_date = '2023-01-31'):
    # 登陆系统
    lg = bs.login()
    # 显示登陆返回信息
    print('login respond error_code:' + lg.error_code)
    print('login respond  error_msg:' + lg.error_msg)

    # 获取沪深A股历史K线数据
    stock_code = code
    stock_start_date = start_date #baostock包可以获取1990年12月19日的所有数据
    stock_end_date = end_date
    if index:
        stock_index = index
    else:
        stock_index = "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST"
    """
    date:  交易日日期
    code:  股票代码
    open:  开盘价
    high:  最高价
    low:   最低价
    close: 收盘价
    preclose: 前收盘价 说明：http://baostock.com/baostock/index.php/A%E8%82%A1K%E7%BA%BF%E6%95%B0%E6%8D%AE
    volume:	成交量（累计 单位：股）	
    amount:	成交额（单位：人民币元）	
    adjustflag: 复权状态(1：后复权， 2：前复权，3：不复权）	
    turn:   换手率 [指定交易日的成交量(股)/指定交易日的股票的流通股总股数(股)]*100%
    tradestatus: 交易状态(1：正常交易 0：停牌）	
    pctChg:	涨跌幅（百分比）	日涨跌幅=[(指定交易日的收盘价-指定交易日前收盘价)/指定交易日前收盘价]*100%
    peTTM:	滚动市盈率	(指定交易日的股票收盘价/指定交易日的每股盈余TTM)=(指定交易日的股票收盘价*截至当日公司总股本)/归属母公司股东净利润TTM
    pbMRQ:	市净率	(指定交易日的股票收盘价/指定交易日的每股净资产)=总市值/(最近披露的归属母公司股东的权益-其他权益工具)
    psTTM:  滚动市销率	(指定交易日的股票收盘价/指定交易日的每股销售额)=(指定交易日的股票收盘价*截至当日公司总股本)/营业总收入TTM
    pcfNcfTTM: 滚动市现率	(指定交易日的股票收盘价/指定交易日的每股现金流TTM)=(指定交易日的股票收盘价*截至当日公司总股本)/现金以及现金等价物净增加额TTM
    isST: 是否ST股，1是，0否
    
    """
    
    rs = bs.query_history_k_data_plus(stock_code, stock_index, start_date = stock_start_date, end_date = stock_end_date, frequency = 'd', adjustflag = '3')
    # frequency：数据类型，默认为d，日k线；d=日k线、w=周、m=月、5=5分钟、15=15分钟、30=30分钟、60=60分钟k线数据，不区分大小写；指数没有分钟线数据；周线每周最后一个交易日才可以获取，月线每月最后一个交易日才可以获取。
    # adjustflag：复权类型，默认不复权：3；1：后复权；2：前复权。已支持分钟线、日线、周线、月线前后复权。 BaoStock提供的是涨跌幅复权算法复权因子，具体介绍见：复权因子简介或者BaoStock复权因子简介。
    print('query_history_k_data_plus respond error_code:'+rs.error_code)
    print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

    #### 打印结果集 ####
    data_list = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    #### 结果集输出到csv文件 ####
    result.to_csv(f".\history_k_data{stock_code} f{start_date}t{end_date}.csv", encoding="gbk", index=False)
    print(result)
    
get_stock_data(start_date="1990-12-19",end_date="2023-10-2")

In [ ]:
stock_dataframe = pd.read_csv(".\history_k_datasz.000002 f1990-12-19t2023-10-2.csv")
stock_dataframe['date'] = pd.to_datetime(stock_dataframe['date'])
stock_dataframe.set_index("date", inplace=True)
# 增加 5日 10日 20日 60日 120日 200日均线
stock_dataframe['mean5'] = stock_dataframe.close.rolling(5).mean()
stock_dataframe['mean10'] = stock_dataframe.close.rolling(10).mean()
stock_dataframe['mean20'] = stock_dataframe.close.rolling(20).mean()
stock_dataframe['mean60'] = stock_dataframe.close.rolling(60).mean()
stock_dataframe['mean120'] = stock_dataframe.close.rolling(120).mean()
stock_dataframe['mean200'] = stock_dataframe.close.rolling(200).mean()
stock_dataframe

## 可视化显示股票收盘价

In [ ]:
fig_close = px.line(stock_dataframe, x=stock_dataframe.index, y="close", 
              template="plotly_dark",  title="股票收盘价",
              hover_data=[stock_dataframe.index,'open','close','high','low'],
              width=970, height=700,
              range_x=['1990-12-19','1993-12-18'])
fig_close.update_xaxes(ticklabelmode="period",minor=dict(ticks="inside", showgrid=True,ticklen=4,  
                     dtick=7*24*60*60*1000,  
                     tick0="1991-01-28"))
fig_close.show()


In [ ]:
fig_candle =  go.Figure(data=[go.Candlestick(x=stock_dataframe.index, # 绘制蜡烛图
                                             open=stock_dataframe.open, high=stock_dataframe.high, 
                                             close=stock_dataframe.close, low=stock_dataframe.low,
                                             name='日线'),
                              # 绘制均线
                              go.Scatter(x=stock_dataframe.index, y=stock_dataframe.mean5, name='MA5'),
                              go.Scatter(x=stock_dataframe.index, y=stock_dataframe.mean10, name='MA10'), 
                              go.Scatter(x=stock_dataframe.index, y=stock_dataframe.mean20, name='MA20'),
                              go.Scatter(x=stock_dataframe.index, y=stock_dataframe.mean60, name='MA60'),
                              go.Scatter(x=stock_dataframe.index, y=stock_dataframe.mean120, name='MA120'),
                              go.Scatter(x=stock_dataframe.index, y=stock_dataframe.mean200, name='MA200'),],
                               layout=go.Layout(title=f"股票{stock_dataframe.code[0]}  K线图",autosize=True))
fig_candle.update_layout(xaxis_range=[datetime.datetime(1990, 12, 19),
                               datetime.datetime(1993, 12, 18)])
fig_candle.show()

In [ ]:
# 增加这段注释测试git filter